# Importing important libraries

In [133]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
import requests
from bs4 import BeautifulSoup
import numpy as np
import re

# Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitar

In [167]:
driver=webdriver.Chrome("chromedriver.exe")
url="https://www.amazon.in/"
driver.get(url)

Navigating to the searchbar and clear any input feed if present

In [168]:
search_bar=driver.find_element_by_xpath('//input[@id="twotabsearchtextbox"]')
search_bar.clear()

In [169]:
search_bar.send_keys("guitars")
search_button=driver.find_element_by_xpath('//input[@id="nav-search-submit-button"]').click()

# Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

Creating empty lists for different attributes

In [151]:
Brand_name=[]
Name_of_product=[] 
No_of_ratings=[]
Price=[]
Return_Exchange=[]
Expected_delivery=[]
Availability=[]
Other_details=[]
Product_url=[]

Creating list of links from which we have to scrape the data

In [170]:
links=["https://www.amazon.in/s?k=guitars&qid=1629550454&ref=sr_pg_1","https://www.amazon.in/s?k=guitars&qid=1629550454&ref=sr_pg_2","https://www.amazon.in/s?k=guitars&qid=1629550454&ref=sr_pg_3"]

Using for loop to add the data into the empty lists

In [119]:
href_links=[]
for i in links:
    driver.get(i)
    element=driver.find_elements_by_xpath("//h2//a[@href]")
    for i in element:
        href_links.append((i.get_attribute("href")))
       
    


In [152]:
for i in href_links:
    driver.get(i)
    try:
         brand=driver.find_element_by_xpath('//a[@id="bylineInfo"]')
         Brand_name.append(brand.text)
    except:
        Brand_name.append("-")
    try:
        name=driver.find_element_by_xpath('//span[@id="productTitle"]')
        Name_of_product.append(name.text)
    except:
        Name_of_product.append("-")
   
    try:
        nu=driver.find_element_by_xpath('//span[@id="acrCustomerReviewText"]')
        No_of_ratings.append(nu.text)
        
    except:
        No_of_ratings.append("-")


In [171]:
 Rating=[]
for i in href_links:
    driver.get(i)
    try:
        rat=driver.find_element_by_xpath('//i//span[@class="a-icon-alt"]')
        Rating.append(rat.text)
    except:
        Rating.append("-")

In [155]:
for i in href_links:
    driver.get(i)
    try:
         pri=driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')
         Price.append(pri.text)
    except:
        Price.append("-")
        
    try:
        exp=driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]')
        Expected_delivery.append(exp.text)
    except:
        Expected_delivery.append("-")
        
    try:
        other=driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        Other_details.append(other.text)
    except:
        Other_details.append("-")
    
        
        
        
        

Creating a dataframe with the help of the lists

In [172]:
guitar=pd.DataFrame({"Brand Name":Brand_name,"Name of Product":Name_of_product,"Rating":Rating,"Number of Ratings":No_of_ratings,"Price":Price,"Expected Delivery":Expected_delivery,"Other details":Other_details})

In [173]:
guitar

Brand Name  \
0              Brand: medellin   
1       Visit the JUAREZ Store   
2      Visit the Kadence Store   
3      Visit the Kadence Store   
4       Visit the JUAREZ Store   
..                         ...   
175               Brand: MAYIS   
176  Visit the blueberry Store   
177     Visit the JUAREZ Store   
178     Visit the JUAREZ Store   
179     Visit the JUAREZ Store   

                                       Name of Product Rating  \
0       Medellin MED-BLU-C Linden Wood Acoustic Guitar          
1    Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...          
2    Kadence Frontier Jumbo Semi Acoustic Guitar Wi...          
3    Kadence Frontier Series, Natural Acoustic Guit...          
4    Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...          
..                                                 ...    ...   
175  Portable Pocket Acoustic Guitar Practice Tool ...          
176  Blueberry B-40 IN, Trussrod Acoustic Guitar Ki...          
177  Juârez JRZ38C/MAH 6 Strings Acoustic Guitar 38...          
178  Juârez JRZ38C/ASH 6 Strings Acoustic Guitar 38...          
179  JUAREZ JRZ38C Right Handed Acoustic Guitar (Su...          

    Number of Ratings      Price  \
0         519 ratings  ₹2,399.00   
1      12,174 ratings  ₹2,649.00   
2         981 ratings          -   
3          74 ratings          -   
4      12,174 ratings  ₹2,649.00   
..                ...        ...   
175                 -  ₹1,748.00   
176        13 ratings  ₹3,599.00   
177       192 ratings  ₹2,290.00   
178       195 ratings  ₹2,290.00   
179     4,095 ratings  ₹2,649.00   

                                     Expected Delivery  \
0                FREE delivery: Friday, Aug 27 Details   
1    FREE delivery: Tuesday, Aug 24 Details\nFastes...   
2                FREE delivery: Friday, Aug 27 Details   
3                FREE delivery: Friday, Aug 27 Details   
4    FREE delivery: Tuesday, Aug 24 Details\nFastes...   
..                                                 ...   
175                 FREE delivery: Sep 11 - 15 Details   
176            FREE delivery: Thursday, Aug 26 Details   
177            FREE delivery: Thursday, Aug 26 Details   
178            FREE delivery: Thursday, Aug 26 Details   
179  FREE delivery: Tuesday, Aug 24 Details\nFastes...   

                                         Other details  
0        Material: Wood\nColour: Blue\nAcoustic Guitar  
1    Black Glossy Finish, Number of Frets: 18, Acou...  
2    QUALITY STRINGS ---The surface is coated with ...  
3    Product Type : Acoustic Guitar\nBrand:Kadence,...  
4    Black Glossy Finish, Number of Frets: 18, Acou...  
..                                                 ...  
175           Warranty not applicable for this product  
176  Material: Top: Spruce wood, Side & Back: - lin...  
177  Number of Frets: 18, Acoustic Guitar with stra...  
178  Number of Frets: 18, Acoustic Guitar with stra...  
179  Number of Frets: 18\nAcoustic guitar with stra...  

[180 rows x 7 columns]